In [1]:
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import pickle

In [2]:
# Load the data from the database
reader = kdb.Reader('knowhere')
emil_H_data = reader.get_dataframe_pivoted(collection='iphone_test',username='emil',sensor='GPS')
emil_H_data = emil_H_data[['GPS (Altitude)','GPS (Latitude)','GPS (Longitude)']]
glen_H_data = reader.get_dataframe_pivoted(collection='iphone_test',username='glen',sensor='GPS')
glen_H_data = glen_H_data[['GPS (Altitude)','GPS (Latitude)','GPS (Longitude)']]

sensor_name,GPS (Altitude),GPS (Latitude),GPS (Longitude)
timestamp,,,
2017-03-09 08:09:29,13.4251708984375,40.7461291132874,-73.8634952903482
2017-03-09 08:09:30,13.4251708984375,40.7461291132874,-73.8634952903482
2017-03-09 08:09:31,13.3884887695312,40.745787131638,-73.8647085708323
2017-03-09 08:09:32,13.1929321289062,40.7457611477382,-73.8648079802039
2017-03-09 08:09:33,14.4685363769531,40.7456887280948,-73.8649902865979
2017-03-09 08:09:34,14.707763671875,40.7456263667352,-73.8651589304897
2017-03-09 08:09:35,15.9894104003906,40.7456879737235,-73.8653693162593
2017-03-09 08:09:36,16.7223205566406,40.7455903664611,-73.8655322604569
2017-03-09 08:25:06,16.1317443847656,40.7454871433235,-73.8656942826452


In [ ]:
glen_H_data

In [3]:
# Scale the data from the DB
emil_H_data = preprocessing.scale(emil_H_data)
glen_H_data = preprocessing.scale(glen_H_data)

In [7]:
# train the model for Emil
emil_H_032017 = AgglomerativeClustering()
emil_H_032017.set_params(n_clusters = 2)
emil_H_032017.fit(emil_H_data)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=2,
            pooling_func=<function mean at 0x11065b848>)

In [8]:
# train the model for Glen
glen_H_032017 = AgglomerativeClustering()
glen_H_032017.set_params(n_clusters = 2)
glen_H_032017.fit(glen_H_data)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=2,
            pooling_func=<function mean at 0x11065b848>)

In [10]:
# Pickle the model for Emil
pickle_emil_H_031717 = pickle.dumps(emil_H_032017)

In [ ]:
# Load the file from the pickle
emil_H_031717 = pickle.loads(pickle_emil_H_032017)

In [11]:
# Pickle the model for Glen
pickle_glen_H_031717 = pickle.dumps(glen_H_032017)

In [ ]:
# Load the file from the pickle
glen_H_031717 = pickle.loads(pickle_glen_H_032017)

In [12]:
# build uploader
uploader = kdb.S3('knowhere')

In [13]:
# Upload models to s3 and db
uploader.upload_to_s3('emil', 'H', emil_H_032017, 'emil_H_032017', 'knowhere-data', 'models')
uploader.upload_to_s3('glen', 'H', glen_H_032017, 'glen_H_032017', 'knowhere-data', 'models')

In [14]:
# Download models from s3
# getting all models for a user
emil_H_model = uploader.retrieve_from_s3('emil', model_type='H', bucket='knowhere-data', collection='models')
# getting a specific model for a user
glen_H_model = uploader.retrieve_from_s3('glen', model_type='H', bucket='knowhere-data', collection='models')

In [11]:
# Makeing Predictions
emil_H_model.fit_predict(emil_H_data)

array([0, 0, 0, ..., 2, 2, 0])

In [12]:
glen_H_model.fit_predict(glen_H_data)

array([1, 1, 1, ..., 1, 1, 1])